In [1]:
import datetime

class Appointment():
    
    def __init__(self, meeting_type, patient, doctor, timeframe):
        self.meeting_type = meeting_type
        self.patient = patient
        self.doctor = doctor
        self.timeframe = timeframe
    
    def __str__(self):
        return(f"Meeting type: {self.meeting_type} , Patient: {self.patient}, Doctor: {self.doctor}, Time: {self.timeframe[0]}-{self.timeframe[1]}")

In [2]:
class Patient():

    def __init__(self, name = "Patient"):
        self.name = name
        self.appointments = []
        self.previous_appointments = []
        
    def __str__(self):
        return f"{self.name}"
    def add_appointment(self, appointment):
        self.appointments.append(appointment)
    
    def complete_appointment(self, appointment):
        if(appointment in self.appointments):
            self.previous_appointments.append(appointment)
            self.appointments.remove(appointment)
    
    def cancel_appointment(self, appointment):
        if(appointment in self.appointments):
            self.appointments.remove(appointment)
    
    def get_appointments(self):
        print(f"Appointment list for Patient: {self.name}")
        for app in self.appointments:
            print(app)

In [3]:
class Doctor():
    def __init__(self, name = "Doctor"):
        self.name = name
        self.appointments = []
        self.unavailable = []
        self.previous_appointments = []
    
    def __str__(self):
        return f"{self.name}"
    def add_appointment(self, appointment):
        self.appointments.append(appointment)
    
    def complete_appointment(self, appointment):
        if(appointment in self.appointments):
            self.previous_appointments.append(appointment)
            self.appointments.remove(appointment)
    
    def cancel_appointment(self, appointment):
        if(appointment in self.appointments):
            self.appointments.remove(appointment)
            
    def get_appointments(self):
        print(f"Appointment list for Doctor: {self.name}")
        for app in self.appointments:
            print(app)

In [4]:
class Schedule():
    
    def __init__(self):
        self.sched = []
    
    def create_appointment(self,meeting_type, patient, doctor, timeframe):
        appointment = Appointment(meeting_type, patient, doctor, timeframe)
        patient.add_appointment(appointment)
        doctor.add_appointment(appointment)
        self.sched.append(appointment)
        self.sched = sorted(self.sched, key = lambda t: t.timeframe[1])
    
    def cancel_appointment(self, appointment):
        if(appointment in self.sched):
            appointment.patient.cancel_appointment(appointment)
            appointment.doctor.cancel_appointment(appointment)
            self.sched.remove(appointment)
    def get_appointments(self):
        print("List of appoints in the schedule: ")
        for app in self.sched:
            print(app)
            
    def check_sched(self, timeframe):
        for app in self.sched:
            if(app.timeframe[0] == timeframe[0] or app.timeframe[1] == timeframe[0] or app.timeframe[0] == timeframe[1] or app.timeframe[1] == timeframe[1]):
                return False
        return True
    
    def clear_sched(self):
        self.sched = []

In [5]:
patients = [Patient("Bob"), Patient("Tom"), Patient("John")]
doctors = [Doctor("Jim"), Doctor("Pam"), Doctor("Sam")]
sched = Schedule()

In [6]:
def create_appointment():
    choice = None
    while(True):
        choice = input("What type of appointment would you like: [Online] [Phone] [Office] ").lower()
        if(choice == "online" or choice == "phone" or choice == "office"):
            break
        print("Not a valid option! ")
    while(True):
        print("List of patients on record: ")
        for patient in patients:
            print(patient)
        person = input("Which patient is this for? If patient not on record, please [Add] a new patient. ")
        try:
            if(person.lower() == "add"):
                add_patient()
            else:
                patient = [p for p in patients if p.name == person]
                if(patient):
                    print(f"Patient Selected: {patient[0]}")
                    break
                else:
                    print("Patient not found! ")
        except:
            print("Patient not found! ")
    while(True):
        print("List of available doctors on record")
        for doc in doctors:
            print(doc)
        doc = input("Which doctor is this for? ")
        try:
            doctor = [d for d in doctors if d.name == doc]
            if(doctor):
                print(f"Doctor selected: {doctor[0]}")
                break
            else:
                print("Doctor not found! ")
        except:
            print("Doctor not found! ")
    while(True):
        try:
            time = input('''What time would you like this appointment? Office appointments take an hour while phone and online appointments take half an hour. Please choose between 09:00 and 17:00 and increments of half hours! [HH:MM] ''')
            time = time.split(":")
            if(len(time) == 2 and 9<=int(time[0])<=17):
                start = datetime.time(int(time[0]), int(time[1]))
                if(choice == "Office"):
                    endtime_h = int(time[0])+1
                    endtime_m = int(time[1])
                else:
                    endtime_h = int(time[0])+round((int(time[1])+30)/60)
                    endtime_m = (int(time[1])+30)%60
                end = datetime.time(int(time[0])+1, int(time[1])) if choice == "Office" else datetime.time(endtime_h,endtime_m)
                if(sched.check_sched((start,end))):
                    break
                else:
                    print("Time slot not available! ")
        except:
            print("Incorrect format for time! ")
    sched.create_appointment(choice, patient[0], doctor[0], (start,end))
    print(f"Appointment created for {patient[0]} with {doctor[0]} at {choice} at time {start} - {end}")

In [7]:
def clear_sched():
    sched.clear_sched()
    print("Schedule has been cleared")

In [8]:
def get_appointments():
    sched.get_appointments()

In [9]:
def add_patient():
    new_patient = input("Name of the new patient: ")
    patients.append(Patient(new_patient))
    print("Patient has been added to record.")

In [10]:
def cancel_appointment():
    for i, v in enumerate(sched.sched):
        print(f"[{i+1}] {v}")
    while(True):
        try:
            choice = int(input("Choose which appointment to cancel or [0] to go back. "))
            if(choice==0):
                break
            else:
                print(f"Appointment {choice} will be deleted.")
                sched.cancel_appointment(sched.sched[choice-1])
                print("Appointment deleted! ")
                break
        except:
            print("Not a valid option!")

In [13]:
from IPython.display import clear_output
def main():
    print("Welcome to the Patient-Doctor Scheduler.")
    while(True):
        begin = input("Would you like to [Get] today's appointments, [Schedule] a new appointment, [Cancel] an appointment, [Add] a new patient, [Clear] today's schedule, or [Nothing]? ").lower()
        if(begin == "get"):
            get_appointments()
        elif(begin == "schedule"):
            create_appointment()
        elif(begin == "cancel"):
            cancel_appointment()
        elif(begin == "add"):
            add_patient()
        elif(begin == "clear"):
            clear_sched()
        elif(begin == "nothing"):
            break
        else:
            print("Not a valid option! ")
main()

Welcome to the Patient-Doctor Scheduler.
Would you like to [Get] today's appointments, [Schedule] a new appointment, [Cancel] an appointment, [Add] a new patient, [Clear] today's schedule, or [Nothing]? get
List of appoints in the schedule: 
Meeting type: online , Patient: Bob, Doctor: Jim, Time: 09:00:00-09:30:00
Would you like to [Get] today's appointments, [Schedule] a new appointment, [Cancel] an appointment, [Add] a new patient, [Clear] today's schedule, or [Nothing]? nothing
